In [1]:
import os
import json
import uuid
import glob 
from tqdm import tqdm
import numpy as np
from PIL import Image
from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.utils.modanetDrawer_test import ModaNetDrawerTest
from maskrcnn_benchmark.utils.viz_utils import *
from PIL import Image
import pycocotools.mask as mask_utils
import matplotlib.pyplot as plt

In [40]:
images = glob.glob('/data/runway2shop/looks/*.jpg')
SAVEDIR = '/data/runway2shop/maskrcnn_preds/looks/'

In [41]:
categories = [
    "__background",
    "bag",
    "belt",
    "outer",
    "dress",
    "sunglasses",
    "pants",
    "top",
    "shorts",
    "skirt",
    "headwear",
    "scarf & tie"
]

In [42]:
cfg.merge_from_file('../configs/imaterialist+modanet-8gpu-COCOft.yaml')
cfg.merge_from_list(['MODEL.WEIGHT', '/data/logs/garment_detection/imaterialist+modanet_continue/model_0047500.pth'])
cfg.merge_from_list(['TEST.IMS_PER_BATCH', '2'])
drawer = ModaNetDrawerTest(cfg, min_image_size=800, confidence_threshold=0.5)

In [43]:
for image_path in tqdm(images) :
    
    image = np.array(Image.open(image_path).convert('RGB'))
    result, top_predictions = drawer.run_on_opencv_image(image)

    bboxes = top_predictions.bbox.numpy().tolist()
    labels = top_predictions.get_field("labels").numpy().astype(int).tolist()
    scores = top_predictions.get_field("scores").numpy().tolist()
    
    masks = top_predictions.get_field("mask").numpy()
    masks = masks.squeeze(1).transpose((1,2,0))
    rle_encoded = mask_utils.encode(np.asfortranarray(masks))
    
    for mask in rle_encoded :
        mask['counts'] = mask['counts'].decode('utf-8')
    
    pred = {
        'labels': labels,
        'scores': scores,
        'bboxes': bboxes,
        'masks': rle_encoded
    }
    
    
    with open(os.path.join(SAVEDIR, os.path.basename(image_path).split('.')[0] + '.json'), 'w') as f :
        json.dump(pred, f)

100%|██████████| 110/110 [01:56<00:00,  1.14s/it]


## Create masked predictions for imaterialist input

lets load the look predictions from maskrcnn and mask the original look images to isolate the garment 

In [114]:
look_images = glob.glob('/data/runway2shop/looks/*.jpg')
look_preds = glob.glob('/data/runway2shop/maskrcnn_preds/looks/*.json')

In [151]:
for img_fn, pred_fn in zip(look_images, look_preds) :
    assert(os.path.basename(img_fn).split('.')[0]==os.path.basename(pred_fn).split('.')[0])
    imgid = os.path.basename(img_fn).split('.')[0]
    img = np.array(Image.open(img_fn).convert('RGB'))
    with open(pred_fn, 'r') as f :
        preds = json.load(f)
        
    num_preds = len(preds['labels'])
    
    for mask in preds['masks'] :
        mask['counts'] = mask['counts'].encode('utf-8')
    masks = mask_utils.decode(preds['masks'])
    
    for pred_ix in range(num_preds) :
        img_ = np.copy(img)
        mask = masks[:, :, pred_ix]
        img_[np.where(mask==0)] = [255, 255, 255]
        bbox = list(map(int, preds['bboxes'][pred_ix]))
        img_ = img_[bbox[1]:+bbox[3], bbox[0]:bbox[2]]
        img_ = Image.fromarray(img_)
        predfn = imgid + '-' + str(pred_ix) + '.jpg'
        img_.save(os.path.join('/data/runway2shop/masked_look_preds', predfn))

In [150]:
img_[np.where(mask==0)].shape

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

## Create masked predictions hautell-looks

let's load all the looks in our database and save the maskrcnn predictions for it including the masked image and bounding box image for easy imaterialist input

In [2]:
images = glob.glob('/data/hautell-looks/**/**/*.jpg')
BASE_DIR = '/data/hautell-looks-garments/'

In [3]:
cfg.merge_from_file('../configs/imaterialist+modanet-8gpu-COCOft.yaml')
cfg.merge_from_list(['MODEL.WEIGHT', '/data/logs/garment_detection/imaterialist+modanet_continue/model_0047500.pth'])
cfg.merge_from_list(['TEST.IMS_PER_BATCH', '8'])
drawer = ModaNetDrawerTest(cfg, min_image_size=800, confidence_threshold=0.5)

In [ ]:
for image_path in tqdm(images) :

    LOOKDIR = os.path.splitext(image_path)[0].split('/')[3:]
    SAVEDIR = os.path.join(BASE_DIR, *LOOKDIR)
    
    if not os.path.exists(SAVEDIR) :
        os.makedirs(SAVEDIR)
        os.makedirs(os.path.join(SAVEDIR, 'bbox_images'))
        os.makedirs(os.path.join(SAVEDIR, 'masked_images'))
    else :
        continue

    image = np.array(Image.open(image_path).convert('RGB'))
    result, top_predictions = drawer.run_on_opencv_image(image)

    # lets gather the predictions 
    
    bboxes = top_predictions.bbox.numpy().tolist()
    labels = top_predictions.get_field("labels").numpy().astype(int).tolist()
    scores = top_predictions.get_field("scores").numpy().tolist()
    
    masks = top_predictions.get_field("mask").numpy()
    masks = masks.squeeze(1).transpose((1,2,0))
    rle_encoded = mask_utils.encode(np.asfortranarray(masks))
    
    for mask in rle_encoded :
        mask['counts'] = mask['counts'].decode('utf-8')
    
    pred = {
        'labels': labels,
        'scores': scores,
        'bboxes': bboxes,
        'masks': rle_encoded
    }
    
    # lets save the prediction
    with open(os.path.join(SAVEDIR, 'predictions.json'), 'w') as f :
        json.dump(pred, f)
        
    # now lets make the masked images
    num_preds = len(bboxes)
    for pred_ix in range(num_preds) :
        img_ = np.copy(image)
        mask = masks[:, :, pred_ix]
        img_[np.where(mask==0)] = [255, 255, 255]
        bbox = list(map(int, bboxes[pred_ix]))
        img_ = img_[bbox[1]:+bbox[3], bbox[0]:bbox[2]]
        img_ = Image.fromarray(img_)
        img_.save(os.path.join(SAVEDIR, 'masked_images', str(pred_ix) + '.jpg'))

    # now lets make the bbox images
    num_preds = len(bboxes)
    for pred_ix in range(num_preds) :
        img_ = np.copy(image)
        bbox = list(map(int, bboxes[pred_ix]))
        img_ = img_[bbox[1]:+bbox[3], bbox[0]:bbox[2]]
        img_ = Image.fromarray(img_)
        img_.save(os.path.join(SAVEDIR, 'bbox_images', str(pred_ix) + '.jpg'))


 37%|███▋      | 6093/16603 [06:18<6:52:57,  2.36s/it]


 38%|███▊      | 6234/16603 [11:42<7:33:03,  2.62s/it]


 38%|███▊      | 6374/16603 [18:29<8:15:15,  2.91s/it]


 39%|███▉      | 6514/16603 [27:15<19:01:42,  6.79s/it]


 40%|████      | 6654/16603 [34:19<8:28:17,  3.07s/it]


...

 42%|████▏     | 6934/16603 [48:13<9:20:30,  3.48s/it]


 43%|████▎     | 7075/16603 [54:23<8:05:34,  3.06s/it]


 43%|████▎     | 7214/16603 [1:02:01<10:02:18,  3.85s/it]


 44%|████▍     | 7349/16603 [1:10:22<6:20:28,  2.47s/it]


 45%|████▍     | 7459/16603 [1:14:54<6:27:10,  2.54s/it]